In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [35]:
train_dir = "/content/drive/MyDrive/Colab Notebooks/braintumor/Training"
test_dir = "/content/drive/MyDrive/Colab Notebooks/braintumor/Testing"

classes = ['glioma', 'meningioma', 'notumor', 'pituitary']
class_labels = {class_name: idx for idx, class_name in enumerate(classes)}

from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

def load_dataset(directory, class_labels, img_size=(128, 128)):
    data, labels = [], []
    for class_name, label in class_labels.items():
        class_dir = os.path.join(directory, class_name)
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            try:
                img = load_img(file_path, target_size=img_size)
                img_array = img_to_array(img) / 255.0
                data.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image {file_path}: {e}")
    return np.array(data), np.array(labels)

img_size = (128, 128)
X_train, y_train = load_dataset(train_dir, class_labels, img_size)
X_test, y_test = load_dataset(test_dir, class_labels, img_size)

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=len(classes))
y_test = to_categorical(y_test, num_classes=len(classes))

print(f"Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}")

Training data shape: (5732, 128, 128, 3), Training labels shape: (5732, 4)
Testing data shape: (1311, 128, 128, 3), Testing labels shape: (1311, 4)


In [36]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

def plot_images(data, labels, num_samples=5):
    plt.figure(figsize=(10, 5))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(data[i])
        plt.title(f"Class: {np.argmax(labels[i])}")
        plt.axis("off")
    plt.show()



In [37]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

print(f"Train shape: {X_train.shape}, Validation shape: {X_val.shape}")

Train shape: (4585, 128, 128, 3), Validation shape: (1147, 128, 128, 3)


In [38]:
    model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(classes), activation='softmax')
])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
batch_size = 32
epochs = 25

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_val, y_val),
    epochs=epochs,
    steps_per_epoch=len(X_train) // batch_size
)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 54/143 ━━━━━━━━━━━━━━━━━━━━ 1:44 1s/step - accuracy: 0.2756 - loss: 2.0936

KeyboardInterrupt: 

In [ ]:
model.save("/content/brain_tumor_model.h5")


In [40]:

from tensorflow.keras.models import load_model
saved_model = load_model("/content/brain_tumor_model.h5")


In [41]:
test_loss, test_accuracy = saved_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

y_pred = np.argmax(saved_model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred, target_names=classes))

conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

41/41 ━━━━━━━━━━━━━━━━━━━━ 10s 239ms/step - accuracy: 0.4525 - loss: 1.5838
Test Accuracy: 63.77%
41/41 ━━━━━━━━━━━━━━━━━━━━ 16s 400ms/step
              precision    recall  f1-score   support

      glioma       0.88      0.32      0.47       300
  meningioma       0.58      0.39      0.47       306
     notumor       0.56      1.00      0.72       405
   pituitary       0.77      0.72      0.75       300

    accuracy                           0.64      1311
   macro avg       0.70      0.61      0.60      1311
weighted avg       0.69      0.64      0.61      1311

Confusion Matrix:
 [[ 97  82  91  30]
 [ 11 120 142  33]
 [  0   2 403   0]
 [  2   2  80 216]]


In [43]:
def predict_image(model, image_path, class_labels):
    img = load_img(image_path, target_size=(128, 128))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = np.argmax(model.predict(img_array))
    return list(class_labels.keys())[list(class_labels.values()).index(prediction)]

test_image_path = "/content/drive/MyDrive/Colab Notebooks/braintumor/Testing/glioma/Te-gl_0100.jpg"
predicted_class = predict_image(saved_model, test_image_path, class_labels)
print(f"The predicted class is: {predicted_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
The predicted class is: notumor
